In [94]:
%load_ext autoreload
%autoreload 2
import notebook_setup
from src.config import PROCESSED_DATA_DIR
import os

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [40]:
from pathlib import Path
FEATURES_DIR = PROCESSED_DATA_DIR / "dataset_129" / 'features'
BREPNET_NPZ_DIR = FEATURES_DIR / "brep"

DINOs14_2d_NPZ_DIR = FEATURES_DIR / '2D' / "dinov2_vits14"
DINOb14_2d_NPZ_DIR = FEATURES_DIR / '2D' / "dinov2_vitb14"
CLIPs16_2d_NPZ_DIR = FEATURES_DIR / '2D' / "ViT-B_16"
CLIPb32_2d_NPZ_DIR = FEATURES_DIR / '2D' / "ViT-B_32"

DINOs14_2dv0_NPZ_DIR = FEATURES_DIR / '2D_v0' / "dinov2_vits14"
DINOb14_2dv0_NPZ_DIR = FEATURES_DIR / '2D_v0' / "dinov2_vitb14"
CLIPs16_2dv0_NPZ_DIR = FEATURES_DIR / '2D_v0' / "ViT-B_16"
CLIPb32_2dv0_NPZ_DIR = FEATURES_DIR / '2D_v0' / "ViT-B_32"

def get_files(step_path: Path, extensions=("stp", "step")):
    return [f for ext in extensions for f in step_path.glob(f"**/*.{ext}")]

brepnet_files = get_files(BREPNET_NPZ_DIR, ('npz',))
print(f"Количество файлов BrepNet: {len(brepnet_files)}")

dino_s14_2d_files = get_files(DINOs14_2d_NPZ_DIR, ('npz',))
dino_b14_2d_files = get_files(DINOb14_2d_NPZ_DIR, ('npz',))
clip_s16_2d_files = get_files(CLIPs16_2d_NPZ_DIR, ('npz',))
clip_b32_2d_files = get_files(CLIPb32_2d_NPZ_DIR, ('npz',))

dino_s14_2dv0_files = get_files(DINOs14_2dv0_NPZ_DIR, ('npz',))
dino_b14_2dv0_files = get_files(DINOb14_2dv0_NPZ_DIR, ('npz',))
clip_s16_2dv0_files = get_files(CLIPs16_2dv0_NPZ_DIR, ('npz',))
clip_b32_2dv0_files = get_files(CLIPb32_2dv0_NPZ_DIR, ('npz',))

print(f"Количество файлов DINO s14 2d: {len(dino_s14_2d_files)}")
print(f"Количество файлов DINO b14 2d: {len(dino_b14_2d_files)}")
print(f"Количество файлов CLIP s16 2d: {len(clip_s16_2d_files)}")
print(f"Количество файлов CLIP b32 2d: {len(clip_b32_2d_files)}")
print(f"Количество файлов DINO s14 2dv0: {len(dino_s14_2dv0_files)}")
print(f"Количество файлов DINO b14 2dv0: {len(dino_b14_2dv0_files)}")
print(f"Количество файлов CLIP s16 2dv0: {len(clip_s16_2dv0_files)}")
print(f"Количество файлов CLIP b32 2dv0: {len(clip_b32_2dv0_files)}")

Количество файлов BrepNet: 129
Количество файлов DINO s14 2d: 129
Количество файлов DINO b14 2d: 129
Количество файлов CLIP s16 2d: 129
Количество файлов CLIP b32 2d: 129
Количество файлов DINO s14 2dv0: 129
Количество файлов DINO b14 2dv0: 129
Количество файлов CLIP s16 2dv0: 129
Количество файлов CLIP b32 2dv0: 129


In [66]:

from src.modeling.vit.metrics import eval_object_max
import numpy as np

def load_npz_features(npz_path):
    data = np.load(npz_path)
    features = data['views']
    return features


model_names = [
    "DINO s14 2d", "DINO b14 2d", "CLIP s16 2d", "CLIP b32 2d",
    "DINO s14 2dv0", "DINO b14 2dv0", "CLIP s16 2dv0", "CLIP b32 2dv0"
]

model_files = [
    dino_s14_2d_files, dino_b14_2d_files, clip_s16_2d_files, clip_b32_2d_files,
    dino_s14_2dv0_files, dino_b14_2dv0_files, clip_s16_2dv0_files, clip_b32_2dv0_files
]

def evaluate_models_for_search(model_names, model_files, load_fn):
    rows = []
    for name, files in zip(model_names, model_files):
        if not files:
            rows.append({"model": name, "queries": 0})
            continue

        m = eval_object_max(files, load_fn, Ks=(1, 5, 10))
        m["model"] = name
        rows.append(m)
    return rows

rows = evaluate_models_for_search(model_names, model_files, load_npz_features)

In [95]:

from src.modeling.BRepNet.metrics import evaluate_brepnet_faces_object_max

out_face_dir = BREPNET_NPZ_DIR / "embeddings_faces"
out_model_dir = BREPNET_NPZ_DIR / "embeddings_models"

brep_metrics = evaluate_brepnet_faces_object_max(
        faces_dir=out_face_dir,
        model_dir=out_model_dir,
        Ks=(1,5,10)
    )

In [96]:
import pandas as pd
df_metrics = pd.DataFrame(rows)
df_brep = pd.DataFrame([{"model": "BRepNet", **brep_metrics}])
# объединяем результаты

df_metrics = pd.concat([df_metrics, df_brep], ignore_index=True)

cols = ["model", "queries", "recall@1", "recall@5", "recall@10", "mAP", "nDCG@5", "nDCG@10",
        "pos_mean", "neg_mean", "margin", "cohens_d"]

df_metrics = df_metrics.reindex(columns=cols)
df_metrics = df_metrics.sort_values([ "margin", "recall@1", "mAP"], ascending=False)
print(df_metrics.to_string(index=False))


        model  queries  recall@1  recall@5  recall@10      mAP   nDCG@5  nDCG@10  pos_mean  neg_mean   margin  cohens_d
DINO s14 2dv0     1032  0.897287  0.996124        1.0 0.943849 0.956899 0.958236       1.0  0.639031 0.360969  2.803236
  DINO b14 2d     1032  0.892442  0.998062        1.0 0.939470 0.954214 0.954904       1.0  0.827749 0.172251  2.222084
DINO b14 2dv0     1032  0.892442  0.998062        1.0 0.939470 0.954214 0.954904       1.0  0.827749 0.172251  2.222084
  DINO s14 2d     1032  0.891473  0.998062        1.0 0.939244 0.954075 0.954766       1.0  0.871053 0.128947  2.246197
      BRepNet     9895  0.826781  0.996564        1.0 0.901286 0.925219 0.926385       1.0  0.876704 0.123296  0.819113
CLIP b32 2dv0     1032  0.891473  0.996124        1.0 0.940215 0.954179 0.955515       1.0  0.894322 0.105678  2.684414
CLIP s16 2dv0     1032  0.901163  0.996124        1.0 0.945706 0.958263 0.959599       1.0  0.906414 0.093586  2.748654
  CLIP s16 2d     1032  0.975775  1.0000

In [102]:
from src.modeling.vit.searcher import vit_topk
query_index = 57
top = 20
# print(vit_topk("DINO s14 2d",  dino_s14_2d_files,  query=query_index, k=top).to_string())
# vit_topk("DINO b14 2d",  dino_b14_2d_files,  query=query_index, k=top)
# vit_topk("CLIP s16 2d",  clip_s16_2d_files,  query=query_index, k=top)
# vit_topk("CLIP b32 2d",  clip_b32_2d_files,  query=query_index, k=top)

print(vit_topk("DINO s14 2dv0", dino_s14_2dv0_files, query=query_index, k=top).to_string())
# vit_topk("DINO b14 2dv0", dino_b14_2dv0_files, query=query_index, k=top)
# vit_topk("CLIP s16 2dv0", clip_s16_2dv0_files, query=query_index, k=top)
# vit_topk("CLIP b32 2dv0", clip_b32_2dv0_files, query=query_index, k=top)

Поисковая модель: DINO s14 2dv0
                               model     score
0    Зацеп трубки направляющий 3.prt  1.000000
1    Зацеп трубки направляющий 4.prt  0.990592
2    Зацеп трубки направляющий 2.prt  0.974641
3    Зацеп трубки направляющий 5.prt  0.974102
4    Зацеп трубки направляющий 6.prt  0.961694
5    Зацеп трубки направляющий 1.prt  0.946608
6      Зацеп трубки направляющий.prt  0.922435
7    Зацеп трубки направляющий 7.prt  0.816310
8               Камера газовая 8.prt  0.770106
9              Камера газовая 10.prt  0.768642
10              Камера газовая 9.prt  0.762634
11   Зацеп трубки направляющий 9.prt  0.747310
12                42. Ejector-03.prt  0.745427
13                42. Ejector-04.prt  0.744396
14                    42. Ejector-02  0.744301
15                   42. Ejector.prt  0.743181
16  Зацеп трубки направляющий 10.prt  0.742906
17                42. Ejector-01.prt  0.742786
18                42. Ejector-05.prt  0.742602
19   Зацеп трубки направляющ